In [1]:
# This is the solution that got 51 points
!wget https://judge.nitro-ai.org/download/roai-2025/onia/2/custom_archive.zip
!unzip /content/custom_archive.zip

In [30]:
# Solution scaffolding for 'Notatia bizantina'
# Feel free to use anything from this
import pandas as pd
import cv2
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import numpy as np
import torch._dynamo
torch._dynamo.disable()

from typing import List, Tuple
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
# Data augmentation transforms
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(10),                       # small rotations
    transforms.RandomAffine(0, translate=(0.1, 0.1)),   # small shifts
    transforms.ColorJitter(brightness=0.2, contrast=0.2),# brightness/contrast
    transforms.ToTensor()
])

eval_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

In [32]:
class DeeperCNN(nn.Module):
    def __init__(self, num_classes):
        super(DeeperCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),   # 48x48 -> 48x48
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),                           # 24x24

            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # 24x24 -> 24x24
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),                           # 12x12

            nn.Conv2d(64, 128, kernel_size=3, padding=1), # 12x12 -> 12x12
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),                           # 6x6

            nn.Conv2d(128, 256, kernel_size=3, padding=1),# 6x6 -> 6x6
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))                  # 1x1
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)  # flatten to (batch, features)
        x = self.classifier(x)
        return x

In [33]:
# CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64*12*12, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64*12*12)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [34]:
# Dataset class
class NeumeDataset(Dataset):
    def __init__(self, csv_path: str, root_dir: str, label_map: dict, train=True):
        self.data = pd.read_csv(csv_path)
        self.root_dir = root_dir
        self.label_map = label_map
        self.train = train

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        path = f"{self.root_dir}/{row['Path']}"
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (48, 48))

        if self.train:
            img = train_transforms(img)
        else:
            img = eval_transforms(img)

        label = self.label_map[row['Effect']]
        return img, torch.tensor(label, dtype=torch.long)

In [35]:
def preprocess_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (48, 48))
    img = img.astype(np.float32) / 255.0  # normalize to [0,1]
    img = np.expand_dims(img, axis=0)     # for channel dimension
    return img

In [53]:
# Utils

def load_data(csv_path: str, root_dir: str) -> Tuple[List, List, dict]:
    df = pd.read_csv(csv_path)
    unique_labels = sorted(df['Effect'].unique())
    label_map = {label: i for i, label in enumerate(unique_labels)}
    return df, label_map

def train_model(train_csv, root_dir):
    df, label_map = load_data(train_csv, root_dir)
    dataset = NeumeDataset(train_csv, root_dir, label_map)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    # model = SimpleCNN(num_classes=len(label_map))
    model = DeeperCNN(num_classes=len(label_map))
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    model.train()
    for epoch in range(100):
        for imgs, labels in dataloader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1} Loss: {loss.item():.4f}")

    return model, label_map

In [37]:
# Process eval images


# Preprocess and extract signs from sequence dataset
def process_sequence_image(model, label_map, path):
    img = cv2.imread(path)
    if img is None:
        print(f"!!! {path}")
        return []

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect bounding boxes for individual neumes
    _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boxes = [cv2.boundingRect(c) for c in contours]
    boxes = sorted(boxes, key=lambda b: b[0])  # left to right

    sequence = []
    inv_label_map = {v: k for k, v in label_map.items()}
    pitch = 0

    model.eval()
    for (x, y, w, h) in boxes:
        neum_img = gray[y:y+h, x:x+w]
        neum_img = cv2.resize(neum_img, (48, 48))
        neum_img = eval_transforms(neum_img).unsqueeze(0).to(device)

        output = model(neum_img)
        pred_label = torch.argmax(output, dim=1).item()
        pred_effect = inv_label_map[pred_label]

        if pred_effect not in ['A', 'B']:
            pitch += int(pred_effect)
        sequence.append(pitch)

    return sequence

In [38]:
# Predict

eval_root = "/content/starting_kit"
# Make predictions and output them to output.csv
def predict(model, label_map):
    results = []
    with open("/content/starting_kit/dataset_eval.csv", "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            image_path = os.path.join(eval_root, row["datapointID"])
            ans_seq = process_sequence_image(model, label_map, image_path)
            results.append({
                "subtaskID": 1,
                "datapointID": row["datapointID"],
                "answer": "|".join(map(str, ans_seq))
            })

    with open("output.csv", "w", encoding="utf-8") as f:
        f.write("subtaskID,datapointID,answer\n")
        for res in results:
            f.write(f"{res['subtaskID']},{res['datapointID']},{res['answer']}\n")

In [54]:
if __name__ == "__main__":
    model, label_map = train_model("/content/starting_kit/dataset_train.csv", "/content/starting_kit")
    predict(model, label_map)
    print("Done")

Epoch 1 Loss: 1.8164
Epoch 2 Loss: 1.5655
Epoch 3 Loss: 1.3639
Epoch 4 Loss: 1.2635
Epoch 5 Loss: 0.9777
Epoch 6 Loss: 0.8921
Epoch 7 Loss: 0.7672
Epoch 8 Loss: 0.5870
Epoch 9 Loss: 0.3922
Epoch 10 Loss: 0.6098
Epoch 11 Loss: 0.3818
Epoch 12 Loss: 0.5902
Epoch 13 Loss: 0.4165
Epoch 14 Loss: 0.2201
Epoch 15 Loss: 0.1628
too long...
Epoch 95 Loss: 0.0014
Epoch 96 Loss: 0.0030
Epoch 97 Loss: 0.0153
Epoch 98 Loss: 0.0489
Epoch 99 Loss: 0.0024
Epoch 100 Loss: 0.0042
Done
